In [50]:
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import pymc3 as pm

import arviz as ar
import matplotlib.pyplot as pl
import seaborn as sb
%matplotlib inline

In [49]:
home = Path.home()

### Methods

The dataset used in this study was generated from the NOMAD dataset. 6 Rrs bands were used at 6 wavelength corresponding to SeaWiFS spectral coverage; 412, 443, 490, 510, 555, 670 nm. The following data transformation steps were implemented:
* invalid data points were flagged and dropped
* for band ratio algorithms, relevant ratios were pre-computed
* for band ratio algorithms with max(Blue) formulation blue band used in the ratio was identified and flagged
* for PCA-based algorithms, principal components were computed for all bands.
* chlorophyll measurement method was identified and flagged
* all predictors were standardized

In [34]:
nomad_file = home / 'DATA/nomad_seabass_v2.a_2008200.txt'
with open(nomad_file) as f:
    for i, line in enumerate(f.readlines()):
        if 'fields' in line:
            columns = line.split('=')[1].split(',')
            continue
        if '/end_header' in line:
            skiprows = i+1

In [36]:
df_raw = pd.read_csv(nomad_file, names=columns, skiprows=skiprows)
bands = [411, 443, 489, 510, 555, 670]

In [39]:
for band in bands:
    df_raw[f'rrs{band}'] = df_raw[f'lw{band}'] / df_raw[f'es{band}']

In [43]:
columns = df_rrs.filter(regex='rrs', axis=1).columns
columns = ['id'] + columns.tolist() + ['chl', 'chl_a']

In [45]:
df = df_raw[columns].copy()

In [46]:
df.head()

,id,rrs411,rrs443,rrs489,rrs510,rrs555,rrs670,chl,chl_a
0,1565,0.001204,0.001686,0.003293,0.004036,0.007479,0.003465,38.19,-999.0
1,1566,0.001062,0.001384,0.002173,0.002499,0.004152,0.001695,35.01,-999.0
2,1567,0.000971,0.001185,0.001843,0.002288,0.004246,0.001612,26.91,-999.0
3,1568,0.001472,0.001741,0.002877,0.003664,0.006982,0.003234,47.96,-999.0
4,1559,0.000905,0.001022,0.001506,0.001903,0.002801,0.001791,23.55,-999.0


In [47]:
df.to_pickle('./df_main.pkl')
df_raw.to_pickle('./df_raw.pkl')

Before a principled statistical comparison of the different chlorophyll algorithms becomes possible, the different algorithms must be re-implemented using a Bayesian framework. The implementation follows the sequence below:

* Model coding using a probabilistic programming language (PPL)
* Prior predictive simulation 
* Model fitting using Hamiltonian Monte Carlo and the Nomad dataset
* Posterior predictive simulation to quantify predictive uncertainty
* Comparison of Model skill  on out-of-sample data using Information Criteria (IC) including Watanabe Akaike Information Criterion (WAIC) and/or Pareto Smoothed Importance Sampling Leave One Out Cross Validation (LOO).


1. Implementation of $OC_4$ 


    A. Without chlorophyll measurement error

    B. With chlorophyll measurement error

2. Implementation of $OC_x$ with $CI$ split


    A. Without chlorophyll measurement error

    B. With chlorophyll measurement error

3. Simple band ratio

    A. Without chlorophyll measurement error

    B. With chlorophyll measurement error

4. Multivariate Linear Regression


    A. Without chlorophyll measurement error

    B. With chlorophyll measurement error

5. Multivariate Linear Regression with Prior Transformation of Principal Components of Reflectance

    A. Without chlorophyll measurement error

    B. With chlorophyll measurement error